In [ ]:
psycopg2

In [1]:
from src.ingestion.database.reader import IngestionReaderInterface
from src.ingestion.database.reader_psql import ConfigurePostgresSparkSession
from src.ingestion.database.reader_psql import PostgresIngestionReader
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import array_contains, col, explode
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector

ModuleNotFoundError: No module named 'psycopg2'

In [6]:
spark_builder = SparkSession.builder.appName("Jupyter")
spark_builder = ConfigurePostgresSparkSession(spark_builder)
spark = spark_builder.getOrCreate()
reader = PostgresIngestionReader(db_host="127.0.0.1", db_user="minfei", db_password="FM199601060046gg", spark=spark)
df = ReadContents(reader)

In [10]:
df.select(["tmdb_primary_info"]).take(1)[0]

# {"id": 399168, "adult": false, "title": "The Mathematician and the Devil", "video": false, "budget": 0, "genres": [{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 14, "name": "Fantasy"}], "status": "Released", "imdb_id": "tt3154916", "revenue": 0, "runtime": 21, "tagline": "", "homepage": "", "overview": "A mathematician offers to sell his soul to the devil for a proof or disproof of Fermat\'s Last Theorem. Based on \\"The Devil and Simon Flagg\\" by Arthur Porges.", "popularity": 0.6, "vote_count": 6, "poster_path": "/5JCaWtCySRPy2JbHwgUAmYJBM8b.jpg", "release_date": "1972-06-06", "vote_average": 8.3, "backdrop_path": null, "original_title": "Математик и чёрт", "spoken_languages": [{"name": "Pусский", "iso_639_1": "ru", "english_name": "Russian"}], "original_language": "ru", "production_companies": [{"id": 88367, "name": "Centrnauchfilm", "logo_path": "/8BGGqyuaxijzhqzmrgdCINWbPhj.png", "origin_country": "SU"}], "production_countries": [{"name": "Soviet Union", "iso_3166_1": "SU"}], "belongs_to_collection": null}'

Row(tmdb_primary_info='{"id": 399168, "adult": false, "title": "The Mathematician and the Devil", "video": false, "budget": 0, "genres": [{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 14, "name": "Fantasy"}], "status": "Released", "imdb_id": "tt3154916", "revenue": 0, "runtime": 21, "tagline": "", "homepage": "", "overview": "A mathematician offers to sell his soul to the devil for a proof or disproof of Fermat\'s Last Theorem. Based on \\"The Devil and Simon Flagg\\" by Arthur Porges.", "popularity": 0.6, "vote_count": 6, "poster_path": "/5JCaWtCySRPy2JbHwgUAmYJBM8b.jpg", "release_date": "1972-06-06", "vote_average": 8.3, "backdrop_path": null, "original_title": "Математик и чёрт", "spoken_languages": [{"name": "Pусский", "iso_639_1": "ru", "english_name": "Russian"}], "original_language": "ru", "production_companies": [{"id": 88367, "name": "Centrnauchfilm", "logo_path": "/8BGGqyuaxijzhqzmrgdCINWbPhj.png", "origin_country": "SU"}], "production_countries": [{"name"

In [142]:
df.select(['tags']).take(1)

[Row(tags='[{"tag": "Bridget Terry", "timestamp_secs": 1422435184}, {"tag": "hollywood", "timestamp_secs": 1526624492}, {"tag": "biography", "timestamp_secs": 1526624492}]')]

In [5]:
import json

json.loads(df.select(['tags']).take(1)[0]["tags"])


[{'tag': 'Bridget Terry', 'timestamp_secs': 1422435184},
 {'tag': 'hollywood', 'timestamp_secs': 1526624492},
 {'tag': 'biography', 'timestamp_secs': 1526624492}]

In [6]:
json.loads(df.select(['tags']).take(1)[0]["tags"])[0]

{'tag': 'Bridget Terry', 'timestamp_secs': 1422435184}

In [7]:
rating_df = ReadRatingFeedbacks(reader)

rating_df.where(rating_df["content_id"] == 1).show()

+-------+----------+-------------------+------+
|user_id|content_id|           rated_at|rating|
+-------+----------+-------------------+------+
|    784|         1|1999-10-13 20:34:04|   3.0|
|   1063|         1|1996-05-19 08:56:11|   4.0|
|   1117|         1|2004-11-12 07:42:09|   4.0|
|   1210|         1|2016-03-11 02:17:21|   5.0|
|     67|         1|2017-11-09 22:33:23|   4.0|
|   1888|         1|2006-09-20 23:20:26|   4.5|
|   1502|         1|2015-10-09 21:03:00|   5.0|
|   2571|         1|2016-08-21 14:03:16|   4.0|
|   1064|         1|2007-01-25 04:21:13|   3.5|
|   1887|         1|1997-01-03 06:20:27|   5.0|
|   1718|         1|1996-07-20 20:46:26|   5.0|
|   2357|         1|2015-08-01 23:25:22|   3.0|
|   2301|         1|2015-08-04 22:17:57|   4.0|
|   4452|         1|1996-05-26 23:44:15|   5.0|
|   4849|         1|2008-11-17 04:20:32|   4.0|
|   2160|         1|2015-07-26 17:45:19|   3.0|
|   5206|         1|2000-08-11 15:03:37|   5.0|
|   5410|         1|2009-06-12 18:22:14|

In [126]:
df_sep.show()

+--------------------+
|       sparse_genres|
+--------------------+
|      (20,[5],[1.0])|
|(20,[1,5],[1.0,1.0])|
|      (20,[0],[1.0])|
|      (20,[8],[1.0])|
|(20,[8,9,15],[1.0...|
|      (20,[0],[1.0])|
|(20,[5,9],[1.0,1.0])|
|(20,[1,17],[1.0,1...|
|     (20,[17],[1.0])|
|(20,[1,3,16],[1.0...|
|      (20,[9],[1.0])|
|(20,[3,15],[1.0,1...|
|     (20,[19],[1.0])|
|      (20,[1],[1.0])|
|(20,[5,15],[1.0,1...|
|(20,[1,2,8],[1.0,...|
|(20,[1,2,5,15],[1...|
|(20,[5,8],[1.0,1.0])|
|(20,[8,15],[1.0,1...|
|(20,[6,8,14],[1.0...|
+--------------------+
only showing top 20 rows



In [134]:
df1 = df.select('id','genres')#.filter(df.id == '124861')
# df1.first()['genres'] ##2 7 8
genres_list = [x[0] for x in df1.select(explode("genres").alias("genres")).distinct().orderBy("genres").collect()]
df_sep = df1.select("*" ,*[
    array_contains("genres", g).alias("g_{}".format(g)).cast("integer")
    for g in genres_list]
).drop('genres')
#df_sep.show()
selected_columns = [column for column in df_sep.columns if column.startswith("g_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_genres')
df_sep = assembler.transform(df_sep).select('id','sparse_genres')
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
res = df_sep.withColumn('genres', sparse_to_array_udf('sparse_genres')).select('id','genres')
res.show()



+------+--------------------+
|    id|              genres|
+------+--------------------+
|156777|[0.0, 0.0, 0.0, 0...|
| 26425|[0.0, 1.0, 0.0, 0...|
|146366|[1.0, 0.0, 0.0, 0...|
|  2996|[0.0, 0.0, 0.0, 0...|
|185931|[0.0, 0.0, 0.0, 0...|
|192859|[1.0, 0.0, 0.0, 0...|
| 47774|[0.0, 0.0, 0.0, 0...|
|115887|[0.0, 1.0, 0.0, 0...|
|168774|[0.0, 0.0, 0.0, 0...|
|167762|[0.0, 1.0, 0.0, 1...|
|160696|[0.0, 0.0, 0.0, 0...|
|177939|[0.0, 0.0, 0.0, 1...|
| 77778|[0.0, 0.0, 0.0, 0...|
|181541|[0.0, 1.0, 0.0, 0...|
|127220|[0.0, 0.0, 0.0, 0...|
|151410|[0.0, 1.0, 1.0, 0...|
| 33679|[0.0, 1.0, 1.0, 0...|
| 33603|[0.0, 0.0, 0.0, 0...|
| 92307|[0.0, 0.0, 0.0, 0...|
|155386|[0.0, 0.0, 0.0, 0...|
+------+--------------------+
only showing top 20 rows




[Stage 1203:>                                                       (0 + 1) / 1]



In [ ]:
df1 = content_genres.select('id','cont')
genres_list = [x[0] for x in df1.select(explode("genres").alias("genres")).distinct().orderBy("genres").collect()]
df_sep = df1.select("*", *[
    array_contains("genres", g).alias("g_{}".format(g)).cast("integer")
    for g in genres_list]
    ).drop('genres')
selected_columns = [column for column in df_sep.columns if column.startswith("g_")] 
assembler = VectorAssembler(inputCols=selected_columns, outputCol='sparse_genres')
df_sep = assembler.transform(df_sep).select('id','sparse_genres')
def sparse_to_array(v):
    v = DenseVector(v)
    new_array = list([float(x) for x in v])
    return new_array

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))
res = df_sep.withColumn('genres', sparse_to_array_udf('sparse_genres')).select('id','genres')
return res